In [1]:
import sys
import os
import subprocess
import shutil
import numpy as np
from datetime import datetime, timedelta

sys.path.append('../sdpm_py_util')
import init_funs_forecast as initfuns
import run_funs as runfuns
sys.path.append('/home/mspydell/models/PFM_root/PFM/driver')
import driver_functions as driverfuns
#from driver_run_forecast_LV1234 import driver_run_fore_LV1234
from driver_run_pfm_phm import driver_run_pfm_phm


/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [ ]:
pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'
runfuns.run_slurm_LV4(pkl_fnm, 'fore')

In [ ]:
infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'


t00 = datetime.now()
# upon initialization, make the model_info.pkl file
initfuns.initialize_model( infile , pkl_fnm )

# print info from pickle file
initfuns.print_initial_model_info( pkl_fnm )


In [ ]:

driver_run_fore_LV1234( infile, pkl_fnm )

# to run from the command line do:
# python driver_run_hind_LV123.py '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py' '/scratch/PHM_Simulations/hindcast_pfile.pkl' > hindcast.out


In [2]:
run = 'hind'
#run = 'fore'
if run == 'hind': # hindcasting works with this newer method (6/10/25). make sure you are in PHM-env
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py'
    pkl_fnm = '/scratch/PHM_Simulations/hindcast_pfile.pkl'
else:
    infile = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
    pkl_fnm = '/scratch/PFM_Simulations/forecast_pfile.pkl'



In [3]:

driver_run_pfm_phm( infile, pkl_fnm )


initializing model and making the info.pkl file.
first, we need all the model information.
from /home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py
we create a dictionary of model info
hindcast
the model_info dictionary is saved to /scratch/PHM_Simulations/hindcast_pfile.pkl
this file dictates how the model is run, where files are saved, etc.
------------------------------------------------------------
                        Model Set Up                        
------------------------------------------------------------
hindcast                                                    
from           : 2024-10-11 00:00                           
to             : 2024-10-13 00:00                           
in a           : 1.0 day chunk                              
atm model      : nam_analysis                               
ocean_model    : hycom_hind_wtide                           
running levels : LV1, LV2, LV3                              
starting hindcast with su

/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


PFM info was edited and resaved
the model_info dictionary was resaved to /scratch/PHM_Simulations/hindcast_pfile.pkl
with properly time stamped output file names


doing a sub simulation from 2024101100 to 2024101200
------------------------------------------------------------
starting LV1
starting LV1 hindcast with subprocess...
getting the hycom data for this simulation...
we need  61  hycom nc files...
the  61  nc files already exist, no need to download.
...done
making the hycom pickle file from all hycom.nc files ...

max and min raw hycom data (iz is top [0] to bottom [39]):
max zeta   =  0.816 m          at  ( it, ilat, ilon)     =  ( 23,  49,  49)
min zeta   = -0.995 m          at  ( it, ilat, ilon)     =  (  7, 203,   0)
max u      =  0.608 m/s        at  ( it, iz, ilat, ilon) =  (  3,  8,  74,  53)
min u      = -0.580 m/s        at  ( it, iz, ilat, ilon) =  (  4,  2,  41,  76)
max v      =  0.585 m/s        at  ( it, iz, ilat, ilon) =  (  7,  4,  36,  90)
min v      = -0.633 

Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions.py", line 9145, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_functions.py", line 3340, in make_all_tmp_pckl_ocnR_files_1hrzeta
    ret1 = subprocess.run(cmd_list )     
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/subprocess.py", line 552, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/subprocess.py", line 1203, in communicate
    self.wait()
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/subprocess.py", line 1266, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/subprocess.py", line 2061, in _wait
    (pid, sts)

KeyboardInterrupt: 

In [ ]:
os.chdir('../driver')
# run hindcast as subprocess...
infile_hind = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py'
pkl_fnm_hind = '/scratch/PHM_Simulations/hindcast_pfile.pkl'
cmd_list_hind = ['python','driver_run_pfm_phm.py',infile_hind,pkl_fnm_hind]
ret_hind = subprocess.Popen(cmd_list_hind)     


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


initializing model and making the info.pkl file.
first, we need all the model information.
from /home/mspydell/models/PFM_root/PFM/sdpm_py_util/phm_model_info_devel.py
we create a dictionary of model info
hindcast
the model_info dictionary is saved to /scratch/PHM_Simulations/hindcast_pfile.pkl
this file dictates how the model is run, where files are saved, etc.
------------------------------------------------------------
                        Model Set Up                        
------------------------------------------------------------
hindcast                                                    
from           : 2024-10-11 00:00                           
to             : 2024-10-13 00:00                           
in a           : 1.0 day chunk                              
atm model      : nam_analysis                               
ocean_model    : hycom_hind_wtide                           
running levels : LV1, LV2, LV3                              
starting hindcast with su

/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


PFM info was edited and resaved
the model_info dictionary was resaved to /scratch/PHM_Simulations/hindcast_pfile.pkl
with properly time stamped output file names


doing a sub simulation from 2024101100 to 2024101200
------------------------------------------------------------
starting LV1
starting LV1 hindcast with subprocess...
getting the hycom data for this simulation...
we need  61  hycom nc files...
the  61  nc files already exist, no need to download.
...done
making the hycom pickle file from all hycom.nc files ...

max and min raw hycom data (iz is top [0] to bottom [39]):
max zeta   =  0.816 m          at  ( it, ilat, ilon)     =  ( 23,  49,  49)
min zeta   = -0.995 m          at  ( it, ilat, ilon)     =  (  7, 203,   0)
max u      =  0.608 m/s        at  ( it, iz, ilat, ilon) =  (  3,  8,  74,  53)
min u      = -0.580 m/s        at  ( it, iz, ilat, ilon) =  (  4,  2,  41,  76)
max v      =  0.585 m/s        at  ( it, iz, ilat, ilon) =  (  7,  4,  36,  90)
min v      = -0.633 

In [2]:
os.chdir('../driver')
# run forecast as subprocess...
infile_fore = '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py'
pkl_fnm_fore = '/scratch/PFM_Simulations/forecast_pfile.pkl'
cmd_list_fore = ['python','driver_run_pfm_phm.py',infile_fore,pkl_fnm_fore]
ret_fore = subprocess.run(cmd_list_fore)     


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


initializing model and making the info.pkl file.
first, we need all the model information.
from /home/mspydell/models/PFM_root/PFM/sdpm_py_util/pfm_input_file.py
we create a dictionary of model info
forecast
the model_info dictionary is saved to /scratch/PFM_Simulations/forecast_pfile.pkl
this file dictates how the model is run, where files are saved, etc.
------------------------------------------------------------
                        Model Set Up                        
------------------------------------------------------------
forecast                                                    
from           : 2025-07-02 00:00                           
to             : 2025-07-07 00:00                           
in a           : 5.0 day chunk                              
atm model      : ecmwf                                      
ocean_model    : hycom_new                                  
running levels : LV1, LV2, LV3, LV4                         
starting forecast with subproce

/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater



starting LV1
starting LV1 forecast with subprocess...
we are going to start clean...
getting PFM info...
cleaning out directories...
now making a new PFM.pkl file.
202507020000

Going to do a PFM forecast from
2025-07-02 00:00:00
to
2025-07-07 00:00:00


getting the hycom forecast time...
this will clean, download new hycom files, then find the forecast date covering the PFM times.
we currently have hycom forecasts starting from:
['2025-06-29', '2025-06-30', '2025-07-01']
checking to see if we are missing any files...
we are missing 542 files from these forecasts.
attempting to get these files ...
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... done
getting <=10 hycom files... don

/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


loading /scratch/PFM_Simulations/LV1_Forecast/Forc/atm_tmp_pckl_file.pkl ...
interpolating to LV4 grid...
... done.
rotating velocities to LV 4 roms directions...
...done.
saving atm to LV4 pkl file...

ATM on roms grid dict saved with pickle.
return code: 0 (0=good)
...done.
this took:
0:00:15.625407


we are now saving ATM LV4 to /scratch/PFM_Simulations/LV4_Forecast/Forc/LV4_ATM_FORCING.nc ...
return code: 0 (0=good)
...done.
this took:
0:00:02.572441


driver_run_forcast_LV4: saving LV4_OCN_BC pickle file
interpolating z with normal loops.
OCN_LV 4 _BC dict saved with pickle to:  /scratch/PFM_Simulations/LV4_Forecast/Forc/BC_LV4_tmp_file.pkl
return code: 0 (0=good)
driver_run_forecast_LV4:  done with writing LV4_OCN_BC.pkl file.
this took:
0:05:11.914043


driver_run_forcast_LV4: saving LV4_OCN_BC netcdf file
OCN_BC dict loaded with pickle

max and min of data in ROMS BC file (iz is bottom [0] to top [39]):
max zeta_north =  0.840 m          at  ( it, ilon)     =  ( 97,  28)
min ze

/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater
Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/swan_functions.py", line 411, in <module>
    globals()[args[1]](*args[2:])    
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/swan_functions.py", line 383, in check_and_move
    time.sleep(dt_sec)  # Check every dt_sec second
    ^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1522, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1408, in run_fore_LV4
    runfuns.run_slurm_LV4(pkl_fnm, 'fore')
  File "/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/run_funs.py", line 826, in run_slurm_LV4
    proc = subprocess

KeyboardInterrupt: 

In [ ]:
MI = initfuns.get_model_info(pkl_fnm)
fname_in = MI['lv1_forc_dir'] + '/' + MI['lv1_ocn_tmp_pckl_file']
#cmd_list = ['python','/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py','make_tmp_hy_on_rom_pckl_files_1hrzeta',fname_in,'urm',pkl_fnm]
cmd_list = ['python','/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py','make_all_tmp_pckl_ocnR_files_1hrzeta',pkl_fnm]
ret1 = subprocess.run(cmd_list )     


In [6]:

print(ret1)

CompletedProcess(args=['python', '/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py', 'make_tmp_hy_on_rom_pckl_files_1hrzeta', '/scratch/PFM_Simulations/LV1_Forecast/Forc/hycom_tmp_pckl_file.pkl', 'urm', '/scratch/PFM_Simulations/forecast_pfile.pkl'], returncode=0)


In [ ]:


print('making dot in dot sb ...')
runfuns.make_LV1_dotin_and_SLURM( pkl_fnm , 'fore' )
print('...done')
print('running roms ...')
runfuns.run_slurm_LV1( pkl_fnm , 'fore')
print('...done')



In [3]:
runfuns.run_slurm_LV2( pkl_fnm , 'fore')

 --- making dot_in and dot_sb --- 
run_slurm_LV2: current directory is now:  /scratch/PFM_Simulations/LV2_Forecast/Run
CompletedProcess(args=['sbatch', '--wait', 'LV2_SLURM.sb'], returncode=0, stdout=b'Submitted batch job 2157\n', stderr=b'')
run_slurm_LV2: run command:  ['sbatch', '--wait', 'LV2_SLURM.sb']
subprocess slurm ran correctly? 0 (0=yes)


CompletedProcess(args=['sbatch', '--wait', 'LV2_SLURM.sb'], returncode=0, stdout=b'Submitted batch job 2157\n', stderr=b'')

In [3]:
driverfuns.run_fore_simulation('LV4',pkl_fnm)
#driverfuns.run_fore_LV4(pkl_fnm)


starting LV4 forecast with subprocess...
we are now putting the fore atm data on the roms LV4 grid...


/home/mspydell/models/PFM_root/PFM/sdpm_py_util/ocn_funs_forecast.py:32: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


loading /scratch/PFM_Simulations/LV1_Forecast/Forc/atm_tmp_pckl_file.pkl ...
interpolating to LV4 grid...
... done.
rotating velocities to LV 4 roms directions...
...done.
saving atm to LV4 pkl file...

ATM on roms grid dict saved with pickle.
return code: 0 (0=good)
...done.
this took:
0:00:15.632728


we are now saving ATM LV4 to /scratch/PFM_Simulations/LV4_Forecast/Forc/LV4_ATM_FORCING.nc ...
return code: 0 (0=good)
...done.
this took:
0:00:02.664909


driver_run_forcast_LV4: saving LV4_OCN_BC pickle file
interpolating z with normal loops.
OCN_LV 4 _BC dict saved with pickle to:  /scratch/PFM_Simulations/LV4_Forecast/Forc/BC_LV4_tmp_file.pkl
return code: 0 (0=good)
driver_run_forecast_LV4:  done with writing LV4_OCN_BC.pkl file.
this took:
0:04:58.974955


driver_run_forcast_LV4: saving LV4_OCN_BC netcdf file
OCN_BC dict loaded with pickle

max and min of data in ROMS BC file (iz is bottom [0] to top [39]):
max zeta_north =  1.184 m          at  ( it, ilon)     =  (  0,   0)
min ze

Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/swan_functions.py", line 408, in <module>
    globals()[args[1]](*args[2:])    
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/models/PFM_root/PFM/sdpm_py_util/swan_functions.py", line 367, in check_and_move
    shutil.copy2(fname_full, fnew)  # Use shutil.copy2 to preserve metadata
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/shutil.py", line 476, in copy2
    copystat(src, dst, follow_symlinks=follow_symlinks)
  File "/home/mspydell/anaconda3/envs/PHM-env/lib/python3.12/shutil.py", line 384, in copystat
    lookup("utime")(dst, ns=(st.st_atime_ns, st.st_mtime_ns),
PermissionError: [Errno 1] Operation not permitted
Traceback (most recent call last):
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_functions.py", line 1522, in <module>
    globals()[args[1]](*args[2:])
  File "/home/mspydell/models/PFM_root/PFM/driver/driver_fun

KeyboardInterrupt: 

In [ ]:


os.chdir('../driver')
cmd_list = ['python','driver_functions.py','run_fore_simulation','LV2',pkl_fnm]
ret1 = subprocess.run(cmd_list)     
 
